In [1]:
ll

total 248
-rw-r--r--  1 zhansu  staff   2351  6 13 21:31 README.md
drwxr-xr-x  5 zhansu  staff    160  6 14 14:47 __pycache__/
-rw-r--r--  1 zhansu  staff   2374  6 14 14:39 config.py
-rw-r--r--  1 zhansu  staff   2245  6 13 21:43 config.pyc
drwxr-xr-x  5 zhansu  staff    160  6 13 21:44 data/
-rw-r--r--  1 zhansu  staff  12740  6 14 14:47 dataset.py
-rw-r--r--  1 zhansu  staff   4418  6 13 21:48 evaluation.py
drwxr-xr-x  3 zhansu  staff     96  6  6 10:24 figure/
drwxr-xr-x  6 zhansu  staff    192  6  6 10:24 model/
-rw-r--r--  1 zhansu  staff  33716  6 14 14:51 pred.txt
-rw-r--r--  1 zhansu  staff  24878  6 27 22:08 qa.ipynb
-rw-r--r--  1 zhansu  staff     57  6 13 21:22 requirements.txt
-rw-r--r--  1 zhansu  staff   6684  6 14 14:47 run.py
-rw-r--r--  1 zhansu  staff    238  6 13 21:22 test.py
-rw-r--r--  1 zhansu  staff   4541  6 27 20:22 未命名.ipynb


In [2]:
import tensorflow as tf
import pandas as pd
import sklearn
import os
# tf.enable_eager_execution()

## 载入数据

In [1]:
data = 'data/wiki'
train_file = os.path.join(data,'train.txt')
test_file = os.path.join(data,'test.txt')
df_train = pd.read_csv(train_file,sep = '\t',names = ['s1','s2','flag'],quoting = 3)
# 将s1和s2合并构成一句话变成文本分类问题
df_train['sentence'] = df_train['s1'] + df_train['s2']
df_test = pd.read_csv(test_file,sep = '\t',names = ['s1','s2','flag'],quoting = 3)
df_test['sentence'] = df_test['s1'] + df_test['s2']
df_test = df_test.sample(frac = 1)


NameError: name 'os' is not defined

## 数据分析

In [10]:

import spacy
from spacy.lang.en import English

nlp = spacy.load('en_core_web_md')

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

## 传统模型

In [4]:

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn import linear_model
# from sklearn.feature_extraction.text import TfidfTransformer
# import numpy as np


# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(df_train['sentence'])
# print('X_train_counts:{}'.format(X_train_counts.shape))
# X_test_counts = count_vect.transform(df_test['sentence'])
# print('X_test_counts:{}'.format(X_test_counts.shape))

# # use the tf_idf Transform
# tfidf_transform = TfidfTransformer()
# tf_idf_train = tfidf_transform.fit_transform(X_train_counts)
# print('tf_idf_train:{}'.format(tf_idf_train.shape))
# tf_idf_test = tfidf_transform.transform(X_test_counts)
# print('tf_idf_test:{}'.format(tf_idf_test.shape))
# # logistic regression
# reg = linear_model.LogisticRegression()
# reg.fit(tf_idf_train,df_train['flag'])

# pred = reg.predict(tf_idf_test)



In [5]:
# # evaluation the map and mrr
# print(pred)
# # print(len(df_test['flag']))
# print(np.mean(pred == df_test['flag']))

# # evaluation the map
# import evaluation

# random_map = np.random.randn(len(pred))
# random_map[random_map < 0.5] = 0
# random_map[random_map >= 0.5] = 1
# print(np.mean(random_map == df_test['flag']))
# print(evaluation.evaluationBypandas(df_test,random_map))

# print(evaluation.evaluationBypandas(df_test,pred))

## 深度模型

### 数据预处理

In [6]:
# 处理数据集合

def gen_test(infile,outfile):
    with open(outfile,'w') as fo:
        for line in open(infile):
            q,a,l = line.strip().split('\t')
            fo.write(q + '\t' + a + '\n')
            
def gen_train(infile, outfile):
    """
    extract question answer pairs which has right answer (label 1) and gen triple
    (question, positive answer, negative answer)
    """

    ############# 生成 q 的 a和label 字典  #############
    with open(infile) as fi:
        qa_pairs = {}
        for line in fi:
            q, a, label = line.split("\t")
            label = int(label)
            if q in qa_pairs:
                qa_pairs[q].append((a, label))
            else:
                qa_pairs[q] = [(a, label)]


    ############# 将label存在1的 q 生成(q, a+, a-)，输出 #############
    with open(outfile, "w") as fo:
        for q in qa_pairs:
            pos_ans = set()
            neg_ans = set()
            for a, label in qa_pairs[q]:
                if label == 1:
                    pos_ans.add(a)
                else:
                    neg_ans.add(a)

            if pos_ans:  # has answer
                for a1 in pos_ans:
                    for a2 in neg_ans:
                        triple = [q, a1, a2]
                        fo.write('\t'.join(triple) + '\n')
                        
gen_test(test_file,os.path.join(data,'test'))
gen_train(train_file,os.path.join(data,'train'))

In [7]:
# get vocab
from collections import Counter
def build_vocab(infile,outfile,max_vocab_size = None):
    tokens = []
    word_dict = {}
    for line in open(infile):
        line = ' '.join(line.strip().lower().split('\t')).split(' ')
        for token in line:
            tokens.append(token)
        
    counter = Counter(tokens)
    
    
#     word_count = counter.most_common(max_vocab_size)
        
    word_dict = {w:e + 2 for e,w in enumerate(list(counter))}
    word_dict['UNK'] = 0
    word_dict['<PAD>'] = 1
#     with open(outfile,'w') as fo:
#         for w,e in enumerate(vocab):
#             word_dict[w] = e + 2
#             fo.write(w + '\n')
    return word_dict

word_dict = build_vocab(train_file,os.path.join(data,'vocab'))
# build vocab_file
# build_vocab(train_file,os.path.join(data,'vocab'))

In [8]:
# load embedding
import numpy as np
def get_embedding(fname,vocab,dim = 100):
        print(fname)
        embeddings = np.random.normal(0,1,size = [len(vocab),dim])
        if fname is None: 
            print("embedding is random")
            return embeddings
            
        else:
            word_vecs = {}
            count = 0
            with open(fname,encoding = 'utf-8') as f:
                i = 0
                for line in f:
                    i += 1
                    if i % 100000 == 0:
                        print ('epch %d' % i)
                    items = line.strip().split(' ')
                    if len(items) == 2:
                        vocab_size, embedding_size = items[0], items[1]
                        print (vocab_size, embedding_size)
                    else:
                        word = items[0]
                        if word in vocab:
                            count += 1
                            embeddings[vocab[word]] = items[1:]
            print('there are {} words can be found in dict'.format(count))
        return embeddings
embedding_file = '../glove.6B.100d.txt'
word_embedding = get_embedding(embedding_file,word_dict,dim = 100)

../glove.6B.100d.txt
epch 100000
epch 200000
epch 300000
epch 400000
there are 30821 words can be found in dict


In [9]:
# get batch_data

train_file = os.path.join(data,'train')
test_file = os.path.join(data,'test')
df_train = pd.read_csv(train_file,sep = '\t',names = ['s1','s2','s2_'],quoting = 3)
df_test = pd.read_csv(test_file,sep = '\t',names = ['s1','s2'],quoting = 3)
batch_size = 64

def triple_pair(row):
    return pd.Series({'s1_id':convert_to_word_ids(row['s1']),'s2_id':convert_to_word_ids(row['s2']),'s2_neg_id':convert_to_word_ids(row['s2_'])})
def point_pair(row):
    return pd.Series({'s1_id':convert_to_word_ids(row['s1']),'s2_id':convert_to_word_ids(row['s2'])})
def chunker(seq,size):
    return (seq[pos:pos + size] for pos in range(0,len(seq),size))

def cut(sentence):
    return sentence.strip().lower().split()
def convert_to_word_ids(sentence,max_len = 40):
    indices = []
    tokens = cut(sentence)
    for word in tokens:
        if word in word_dict:
            indices.append(word_dict[word])
        else:
            continue
    result = indices + [word_dict['UNK']] * (max_len - len(indices))

    return result[:max_len]
    
    
def batch_iter_pandas_train(df,batch_size):
    
#     shuffle the dataset
    df = df.sample(frac = 1).reset_index(drop = True)     
    batches = chunker(df,batch_size)
    
    for b in batches:
        q = b['s1_id']
        a = b['s2_id']
        a_ = b['s2_neg_id']
        yield(np.array(q.tolist()),np.array(a.tolist()),np.array(a_.tolist()))
def batch_iter_pandas_test(df,batch_size):
    batches = chunker(df,batch_size)
    
    for b in batches:
        q = b['s1_id']
        a = b['s2_id']
        yield(np.array(q.tolist()),np.array(a.tolist()))
        
# Convert the sentence to ids
df_train_pairs = df_train.apply(triple_pair,axis = 1)

batch_train = batch_iter_pandas_train(df_train_pairs,batch_size)
for q,a,a_ in batch_train:
    print(q)
    break

[[ 514  134  555 ...    0    0    0]
 [   2  865 3204 ...    0    0    0]
 [ 546  134   17 ...    0    0    0]
 ...
 [1679  530  134 ...    0    0    0]
 [1679    3 3752 ...    0    0    0]
 [ 514   20  459 ...    0    0    0]]


### 深度模型

In [10]:
# 利用place_holder

q = tf.placeholder(tf.int32,[None,None],name = 'q')
a = tf.placeholder(tf.int32,[None,None],name = 'a')
a_ = tf.placeholder(tf.int32,[None,None],name = 'a_')


margin = 0.05
learning_rate = 0.001
embedding_size = 100
filter_sizes = [3,4,5]
num_filters = 64
# get embedding

embed = tf.Variable(word_embedding,trainable = False,name = 'fix_embedding')

# get the embed of the sentence

q_embed = tf.nn.embedding_lookup(embed,q)
a_embed = tf.nn.embedding_lookup(embed,a)
a_neg_embed = tf.nn.embedding_lookup(embed,a_)

# # encode the sentence
# q_encode = tf.reduce_sum(q_embed,1,name = 'q_encode')
# a_encode = tf.reduce_sum(a_embed,1,name = 'a_encode')
# a_neg_encode = tf.reduce_sum(a_neg_embed,1,name = 'a_neg_encode')

# convolution the sentence
q_embed_expand = tf.expand_dims(q_embed,-1,'q_expand')
a_embed_expand = tf.expand_dims(a_embed,-1,'a_expand')
a_neg_embed_encode = tf.expand_dims(a_neg_embed,-1,'a_neg_expand')



output_q = []
output_a = []
output_a_neg = []
for filter_size in filter_sizes:
    with tf.name_scope('conv-pool-%s' % filter_size):
    # [batch,sequence_len,embedding,1]
        conv1 = tf.layers.conv2d(q_embed_expand,num_filters,strides = [1,1],
                            kernel_size = [filter_size,embedding_size],
                            padding = 'VALID',
                            reuse = False,
                            activation = tf.nn.relu,
                            name = 'conv_{}'.format(str(filter_size)))


        pool1 = tf.reduce_mean(conv1,1,name = 'pool1',keep_dims=True)
        output_q.append(pool1)
                                 
        conv2 = tf.layers.conv2d(a_embed_expand,num_filters,strides = [1,1],
                            kernel_size = [filter_size,embedding_size],
                            padding = 'VALID',
                            reuse = True,
                            activation = tf.nn.relu,
                            name = 'conv_{}'.format(str(filter_size)))

        pool2 = tf.reduce_mean(conv2,1,name = 'pool2',keep_dims=True)
        
        output_a.append(pool2)


        conv3 = tf.layers.conv2d(a_neg_embed_encode,num_filters,strides = [1,1],
                                kernel_size = [filter_size,embedding_size],
                                padding = 'VALID',
                                reuse = True,
                                activation = tf.nn.relu,
                                name = 'conv_{}'.format(str(filter_size)))

        pool3 = tf.reduce_mean(conv3,1,name = 'pool3',keep_dims=True)
        output_a_neg.append(pool3)

num_filters_total = len(filter_sizes) * num_filters

h_pool_q = tf.concat(output_q,3)
h_pool_a = tf.concat(output_a,3)
h_pool_a_neg = tf.concat(output_a_neg,3)

encode_q = tf.reshape(h_pool_q,[-1,num_filters_total])
encode_a = tf.reshape(h_pool_a,[-1,num_filters_total])
encode_a_neg = tf.reshape(h_pool_a_neg,[-1,num_filters_total])                                 
                                 
                            
                            
# create the loss

def consin_score(q,a):
    q_norm = tf.nn.l2_normalize(q,1)
    a_norm = tf.nn.l2_normalize(a,1)
    
    return tf.cast(tf.reduce_sum(tf.multiply(q_norm,a_norm),1),tf.float32)
score_q_a = consin_score(encode_q,encode_a)
score_q_neg = consin_score(encode_q,encode_a_neg) 

loss = tf.reduce_mean(tf.math.maximum(0.0,tf.math.subtract(margin,tf.math.subtract(score_q_a,score_q_neg))))

correct = tf.equal(0.0,loss)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32),name = 'acc')

# create the train_op

global_step = tf.Variable(0, name = "global_step", trainable = False)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(grads_and_vars, global_step = global_step)





Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use tf.cast instead.


### 训练模型

In [11]:
epochs = 5

sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(epochs):
    
    for s1,s2,s2_ in batch_train:

        feed_dict = {
            q:s1,
            a:s2,
            a_:s2_
        }
        _,step,losses,pos_score,neg_score = sess.run([train_op,global_step,loss,score_q_a,score_q_neg],feed_dict = feed_dict)
        print('loss:{} pos:{} neg:{}'.format(losses,np.mean(pos_score),np.mean(neg_score)))
        
        


loss:0.1304152011871338 pos:0.8110812902450562 neg:0.8412067890167236
loss:0.09701573848724365 pos:0.9175577163696289 neg:0.9515644907951355
loss:0.05771539360284805 pos:0.9826871156692505 neg:0.9903937578201294
loss:0.05463765561580658 pos:0.983115553855896 neg:0.9877532124519348
loss:0.058633144944906235 pos:0.9772401452064514 neg:0.985873281955719
loss:0.05442947521805763 pos:0.9853833317756653 neg:0.9892674088478088
loss:0.05454438179731369 pos:0.9846780300140381 neg:0.9892224073410034
loss:0.05242455378174782 pos:0.9912303686141968 neg:0.9936549663543701
loss:0.052488237619400024 pos:0.9915428161621094 neg:0.9940310716629028
loss:0.051095928996801376 pos:0.9932905435562134 neg:0.9943863749504089
loss:0.05204382911324501 pos:0.9943829774856567 neg:0.9964268207550049
loss:0.052139781415462494 pos:0.9953771829605103 neg:0.9975168704986572
loss:0.050998322665691376 pos:0.9961888790130615 neg:0.9971871376037598
loss:0.05134999006986618 pos:0.9963234663009644 neg:0.997673511505127
loss:

loss:0.04980599135160446 pos:0.9986735582351685 neg:0.9984794855117798
loss:0.05006594583392143 pos:0.9987421035766602 neg:0.9988080263137817
loss:0.04929134249687195 pos:0.9990805983543396 neg:0.9983718395233154
loss:0.049870509654283524 pos:0.9989293217658997 neg:0.9987998604774475
loss:0.04913055896759033 pos:0.9991165399551392 neg:0.9982471466064453
loss:0.049807026982307434 pos:0.998977541923523 neg:0.9987846612930298
loss:0.05032399296760559 pos:0.9979656934738159 neg:0.9982895851135254
loss:0.0504387728869915 pos:0.9977156519889832 neg:0.9981544017791748
loss:0.05033506453037262 pos:0.9978708028793335 neg:0.9982057809829712
loss:0.050032515078783035 pos:0.9984179735183716 neg:0.9984504580497742
loss:0.04976886510848999 pos:0.9988811016082764 neg:0.9986499547958374
loss:0.04980524629354477 pos:0.9989550709724426 neg:0.9987603425979614
loss:0.04984576255083084 pos:0.9990796446800232 neg:0.9989253282546997
loss:0.049390628933906555 pos:0.9992243051528931 neg:0.9986149072647095
loss

### 测试集预测

In [12]:
df_test_pairs = df_test.apply(point_pair,axis = 1)

batch_test = batch_iter_pandas_test(df_test_pairs,100)

test_score = []
for s1,s2 in batch_test:
    feed_dict = {
        q:s1,
        a:s2
    }
    score = sess.run(score_q_a,feed_dict = feed_dict)
    test_score.extend(score)
    

### 评估模型

In [13]:
import evaluation

test_raw_file = os.path.join(data,'test.txt')
df_raw_test = pd.read_csv(test_raw_file,sep = '\t',names = ['s1','s2','flag'],quoting = 3)
assert len(df_test) == len(test_score)
print(evaluation.evaluationBypandas(df_raw_test,test_score))
print(evaluation.evaluationBypandas(df_raw_test,np.random.randn(len(df_raw_test))))

(0.3655099485131332, 0.36623011047365434)
(0.4033543716323824, 0.4097187726542667)
